In [48]:
import numpy as np
import autograd.numpy as anp

In [45]:
x_mean = np.array(([[0.0], [0.0]]), dtype=np.float)
x_sigma =np.array([[1, 0],[ 0, 1]])

In [3]:
np.linalg.cholesky(x_sigma)

array([[ 1.,  0.],
       [ 0.,  1.]])

In [4]:
def get_sigma_points(x_mean, x_sigma, n = 2, alpha= 0.5, beta = 2, kappa = 10):
    sigma_points = np.zeros([n, 2 * n + 1], dtype=np.float)
    
    # Use Cholesky as proxy for square root of the matrix
    L = np.linalg.cholesky(x_sigma)
    
    par_lambda = alpha * alpha * (n + kappa) - n
    sqrt_n_plus_lambda = np.sqrt(n + par_lambda)
    
    sigma_points[:, 0:1] = x_mean
    sigma_points[:, 1:n+1] = x_mean + sqrt_n_plus_lambda * L 
    sigma_points[:, n+1:2*n+1] = x_mean - sqrt_n_plus_lambda * L 
    
    w_m = np.zeros([1, 2 * n + 1 ], dtype=np.float)
    w_c = np.zeros([1, 2 * n + 1 ], dtype=np.float)
    
#     print(par_lambda)
    
    # weights w_m are for computing mean and weights w_c are
    # used for covarince calculation
    
    w_m = w_m + 1 / (2* (n + par_lambda))
    w_c = w_c + w_m
    w_m [0, 0] = par_lambda / (n + par_lambda) 
    w_c [0, 0] = w_m[0,0] + (1 - alpha * alpha + beta)
    
    return sigma_points, w_m, w_c
    
    

In [43]:
def f(x):
    x = np.array(x, dtype=np.float)
    x2 = 2 * x + 1
    return np.sin(x2)

In [38]:
def unscented_transform(f, x_mean, x_sigma):
    n = 2
    sigma_points, w_m, w_c = get_sigma_points(x_mean, x_sigma, n=n)
    
    transformed_points = f(sigma_points)
    pred_mean = np.sum (np.multiply( transformed_points , w_m ) , axis=1)
    pred_mean = pred_mean.reshape([n, 1])
    gofx_minus_mean = f(sigma_points) - pred_mean
    p_s = np.matmul( gofx_minus_mean, np.transpose(gofx_minus_mean))
    #pred_sigma = np.sum(np.multiply( np.matmul( gofx_minus_mean, gofx_minus_mean.transpose()) , w_m ))
    return pred_mean, p_s

In [46]:
sigma_points, w_m, w_c = get_sigma_points(x_mean, x_sigma)

1.0


In [40]:
sigma_points

array([[ 0.        ,  1.73205081,  0.        , -1.73205081,  0.        ],
       [ 0.        ,  0.        ,  1.73205081,  0.        , -1.73205081]])

In [41]:
sigma_list = sigma_points.T.tolist()

In [44]:
list(map(f, sigma_list))

[array([ 0.84147098,  0.84147098]),
 array([-0.96933471,  0.84147098]),
 array([ 0.84147098, -0.96933471]),
 array([-0.62684015,  0.84147098]),
 array([ 0.84147098, -0.62684015])]

In [19]:
unscented_transform(f, x_mean, x_sigma)

1.0


(array([[ 1.],
        [ 1.]]), array([[ 24.,   0.],
        [  0.,  24.]]))

In [154]:
f(x_mean)

array([[ 1.],
       [ 1.]])

In [21]:
x_mean

array([[0],
       [0]])

In [47]:
!pip install autograd

    100% |████████████████████████████████| 829kB 657kB/s ta 0:00:01
  Running setup.py bdist_wheel for autograd ... done
  Stored in directory: /Users/Pontragyin/Library/Caches/pip/wheels/00/f9/b8/3a184a1c17c0d144c43b860e9abb7657ec2d8e03e07c15853a
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /Users/Pontragyin/Library/Caches/pip/wheels/c2/50/7c/0d83b4baac4f63ff7a765bd16390d2ab43c93587fac9d6017a
Successfully built autograd future
